In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install Flask

In [ ]:
%cd /kaggle/working

In [ ]:
!git clone https://github.com/NVIDIA/apex
!cd apex
!pip install -v --no-cache-dir  /kaggle/working/apex

In [ ]:
%%writefile requirements.txt
matplotlib==2.1.0
inflect==0.2.5
Unidecode==1.0.22
pillow
tensorflow==1.15.2
librosa==0.8.1
numba==0.48.0

In [ ]:
!pip install -r requirements.txt --user

In [ ]:
!pip install pyngrok==4.1.1
!ngrok authtoken '281CNui5X6FuaLQ6hPCRMcVBTjr_48bsTy1xokX5ubxn3n2u3'
!pip install SoundFile

In [1]:
%cd /kaggle/input/datasettacotron2flask/flask

In [2]:
from flask import Flask , request
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
# from helper import load_model , tts , pts
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import torch
import librosa
import threading
from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
import re
import sys
sys.path.append('waveglow/')
# matplotlib inline
from denoiser import Denoiser
from unicodedata import normalize
from datetime import datetime
import soundfile as sf

In [30]:
model = None
hparams = None
waveglow = None
denoiser = None
duration_of_char = 1574.5753424657535

app = Flask(__name__)
run_with_ngrok(app) 

@app.route('/')
def index():
	return '<h1>Hello World!</h1>'

@app.route("/text2speech", methods=["POST"])
def text2speech():
    if request.method == "POST":
      now = datetime.now()
      dt_string = now.strftime("%d-%m-%Y-%H:%M:%S")
      file_path = '/kaggle/working/audio_' + dt_string + '.wav'
      data = request.get_json()
      audio = pts(data["text"])
      sf.write('/kaggle/working/audio_' + dt_string + '.wav',audio.T, 22050)
      return "OK"

def load_data():
    global model 
    global hparams
    global waveglow
    global denoiser

    hparams = create_hparams()
    hparams.sampling_rate = 22050

    checkpoint_path = "/kaggle/input/datasettacotron2flask/flask/output_vi/checkpoint_421000"
    model = load_model(hparams)
    model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
    _ = model.cuda().eval().half()

    # waveglow_path = '/content/drive/MyDrive/flask/waveglow/waveglow_256channels_universal_v5.pt'
    # waveglow = torch.load(waveglow_path)['model']
    waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')
    # waveglow_path = '/content/drive/MyDrive/Output/nvidia_waveglowpyt_fp32_20190427'
    # ckpt = torch.load(waveglow_path)
    # state_dict = ckpt['state_dict']
    # config = ckpt['config']
    # waveglow = WaveGlow(**config)
    waveglow.cuda().eval().half()
    for k in waveglow.convinv:
        k.float()
    denoiser = Denoiser(waveglow)


def tts(text):
	text = normalize("NFC", text).lower()
	sequence = np.array(text_to_sequence(text, ['basic_cleaners']))[None, :]
	sequence = torch.autograd.Variable(
		torch.from_numpy(sequence)).cuda().long()
	with torch.no_grad():
		mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
		audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
	audio_denoised = denoiser(audio, strength=0.005)[:, 0].cpu().numpy()
    
	audio_result = audio_denoised
	duration_expected = int(duration_of_char * len(text))
	if (abs(np.size(audio_denoised) - duration_expected) / duration_expected) > 0.25:
		audio_result = np.array([audio_denoised[0][:int(duration_expected)]])
    
	return audio_result


def pts(para):
	audio = np.zeros((1,0))
	sentence_ls = para.split(".")
	for sen in sentence_ls:
		sub_stn_ls = re.split(",|;|-|:", sen)
		for sub_stn in sub_stn_ls:
			audio = np.append(audio, tts(sub_stn), axis=1)
			audio = np.append(audio, np.zeros((1, int(hparams.sampling_rate/8)), dtype=np.uint8) , axis=1)
		audio = np.append(audio, np.zeros((1, int(hparams.sampling_rate/4)), dtype=np.uint8) , axis=1)
	return audio

load_data()
print(model)
app.run()